# Smartware Scholar (sw-scholar)
*Author: Dino Paulo R. Gomez*

Full rewrite of `TranslateAutoChecker`, a script that checks for all missing translations in the **HRSD** Application

In [ ]:
import re
import pickle
import json
import os
import yaml
import glob
import xml.etree.ElementTree as ET
import lxml.etree
from pprint import pprint


# version = '2023.05.1'
# filename = f'lib\{module}\{version}\\app.js'

# Path
module_path = r"C:\Infor\ERC\Patterns\SW4\UiPatterns"
lib_path = r"lib"



allowed_version = "11.1.0"
allowed_modules_prefix = ["AskHR", "CM"]

Here we open and parse our **CSV** file and store it to `word_list=[]`

In [ ]:
word_list = []


def sanitize(input):
    return input.replace('"', "").strip()

csv_filename = 'CMMain'
csv_filepath = f'C:\Infor\AutomatedTranslationChecker\csv\{csv_filename}.csv'
try:
    with open(csv_filepath) as csv_file:
        for line in csv_file:
            if line != "N/A":
                word_list.append(sanitize(line))
    prev = len(word_list)
    print(f"word_list loaded with [{prev} Elements]")
    word_list = set(word_list)
    print(f"word_list removed duplicate [{prev-(len(word_list))} Elements]")
    print("word_list current length",len(word_list), "elements")
except Exception as e:
    print(e)

After we loaded our **CSV** file into our list, we will check if an `resource-dictionary.json` file is existing if not then we will generate it.

`build_file_tree()` is used to map out the directory tree of the **ERC** Folder to deep scan any `resources.xml`

`handle_xml()` is used to parse the **XML** Files

`get_resources_xml()` is used to map out the **XML TREE** inside the scanned xml files


In [ ]:
resource_directories = {}
module_files_path_list = []


def build_file_tree(module_files_path_list):
    check_file = 'scholar-module-list'
    module_files_path_list = module_files_path_list
    if os.path.isfile(check_file):
        print('Scholar: scholar-module-list detected.')
        with open(check_file, "rb") as fp:
            module_files_path_list = pickle.load(fp)
            print('Scholar: scholar-module-list loaded.')
            print(f'Scholar: loaded module_files_path_list with {len(module_files_path_list)} entries')

    else:
        print('Scholar: no scholar-module-list detected, generating new list. ')
        # Get Resource/Resources Directory from UIPatterns
        for (dirpath, dirnames, filenames) in os.walk(module_path):
            for dir in dirnames:
                # Case Check due to folders being inconsistent
                if dir.endswith("resource") or dir.endswith("resources"):
                    for module in allowed_modules_prefix:
                        if module in dirpath:
                            resource_directories[dirpath] = os.sep.join(
                                [dirpath, dir])

        # Filter Directories to store only v11.1.0 resources
        for key, value in list(resource_directories.items()):
            if allowed_version not in key:
                del resource_directories[key]

        # Iterate through Resources List
        for key, path in list(resource_directories.items()):
            for dirpath, dirs, files in os.walk(path):
                for filename in files:
                    file = os.path.join(dirpath, filename)
                    module_files_path_list.append(file)
        with open(check_file, "wb") as fp:
            pickle.dump(module_files_path_list, fp)
            print(f'Scholar: scholar-module-list generated with {len(module_files_path_list)} entries. \n')


    return module_files_path_list


module_files_path_list = build_file_tree(module_files_path_list)
# pprint(module_files_path_list)

In [ ]:
xml_generated_list = []
resource_dictionary = {}


def handle_xml(file):
    temp_dict = {

    }
    tree = ET.parse(file)
    root = tree.getroot()
    root_code = str(root.attrib["code"])
    root_ref = ""
    for map_tag in root.iter("mapping"):
        mapping_dict = {
            'value': '',
            'ref': ''
        }
        for map_item in map_tag:
            mapping_dict['value'] = str(map_item.text)
            root_ref = ""
            key_value = f'"{root_code}.{str(map_tag.attrib["name"])}" : "{str(map_item.text)}"'
            try:
                root_ref = str(map_tag.attrib["ref"])

                if root_ref:
                    mapping_dict['ref'] = str(root_ref)
                else:
                    mapping_dict['ref'] = ''
            except:
                mapping_dict['ref'] = ''
                xml_generated_list.append(key_value)

            temp_dict[str(map_tag.attrib["name"])] = mapping_dict
    if (temp_dict):
        resource_dictionary[root_code] = temp_dict


def get_resources_xml(resource_dictionary):
    global xml_generated_list
    resource_dictionary = resource_dictionary
    check_file = 'scholar-resource-dictionary'
    check_xml_file = 'scholar-xml-generated-list'
    if os.path.isfile(check_file):

        with open(check_file, "rb") as fp:
            resource_dictionary = pickle.load(fp)
            print(f'Scholar: {check_file} detected.')
            print(
                f'Scholar: resource_dictionary loaded with {len(resource_dictionary)} entries.')
        with open(check_xml_file, "rb") as fp:
            xml_generated_list = pickle.load(fp)
            print(f'\nScholar: {check_xml_file} detected.')
            print(
                f'Scholar: xml_generated_list loaded with {len(xml_generated_list)} entries.')

    else:
        for file_path in module_files_path_list:
            handle_xml(file_path)
        with open(check_xml_file, "wb") as fp:
            pickle.dump(xml_generated_list, fp)
            print(f'Scholar: {check_xml_file} generated.')
        with open(check_file, "wb") as fp:
            pickle.dump(resource_dictionary, fp)
            print(f'Scholar: {check_file} generated.')

    return resource_dictionary


resource_dictionary = get_resources_xml(resource_dictionary)
with open('scholar-resource_dictionary.json', 'w') as file:
    file.write(json.dumps(resource_dictionary, indent=4))
    print(
        f'\nScholar: scholar-resource-dictionary.json generated with {len(xml_generated_list)} elements for reference.')

In [ ]:
lib_tree = []
def build_lib_tree(lib_path):
    global lib_tree
    for subdir, dirs, files in os.walk(lib_path):
        for dir in dirs:
            parent_path = os.path.join(lib_path,dir)
            for subdir, dirs, files in os.walk(parent_path):
                if(len(dirs)>0):
                    lib_tree.append(os.path.join(parent_path,dirs[-1],'app.js'))
build_lib_tree('lib')
print(f"lib_tree[] - {len(lib_tree)} app.js detected.\n")
# pprint(lib_tree)

In [ ]:
app_translation_dict = {}


def parse_app_js():

    global app_translation_dict
    check_file = 'scholar-app-translation-dict'
    # Regex Pattern
    ext_regex_pattern = r'Ext\.define\(".*?",\s*{\s*extend:\s*".*?",\s*raw:\s*\{[\s\S]*?\}\);'
    namespace_regex_pattern = r'^Ext\.define\("(.+?)"'
    raw_regex_pattern = r'raw\s*:\s*\{(?:[^{}]+|\{(?:[^{}]+|\{(?:[^{}]+|\{(?:[^{}]+|\{(?:[^{}]+|\{[^{}]*\})*\})*\})*\})*\})*},'
    raw_match_regex_pattern = r'{(?:[^{}]*\{[^{}]*\}[^{}]*)*}'
    if os.path.isfile(check_file):
        with open(check_file, "rb") as fp:
            app_translation_dict = pickle.load(fp)
            print(f'Scholar: {check_file} detected.')
            print(f'Scholar: app_translation_dict loaded with {len(app_translation_dict)} entries.')
    else:
        for count, item in enumerate(lib_tree):
            # # Open the file for reading
            try:
                with open(item, 'r', encoding='utf-8') as f:
                    print(f'Scholar: Opening {item}')
                    # Read the file contents
                    contents = f.read()
                    # Search for matches of the pattern in the file contents
                    matches = re.findall(ext_regex_pattern, contents)
                    namespace_code = ''
                    if matches:
                        for match in matches:
                            # Get the namespace from the Ext.Define Syntax
                            namespace_match = re.search(
                                namespace_regex_pattern, match)
                            namespace_code = namespace_match.group(1)

                            raw_obj_match = re.findall(
                                raw_match_regex_pattern, match)
                            if raw_obj_match:
                                raw_data = raw_obj_match[0]
                                raw_data = raw_data.replace(",\n  }", "\n  }")
                                ndata = yaml.load(raw_data, yaml.SafeLoader)
                                app_translation_dict[namespace_code] = ndata
            except Exception as e:
                print(e)
        with open(check_file, "wb") as fp:
            pickle.dump(app_translation_dict, fp)
            print(f'\nScholar: {check_file} generated with {len(app_translation_dict)} elements.')


parse_app_js()
with open('scholar-app_translation_obj.json', 'w') as file:
    file.write(json.dumps(app_translation_dict, indent=4))
    print(f'\nScholar: scholar-app_translation_obj.json generated with {len(app_translation_dict)} elements for reference.')

In [ ]:
def modify_xml(file,mapping,new_value):

    parser = lxml.etree.XMLParser(strip_cdata=False)
    tree = lxml.etree.parse(file, parser)
    root = tree.getroot()

    # Find the mapping element with a name attribute of "emptytextagent"
    mapping_element = root.find(f".//mapping[@name='{mapping}']")

    # Add a new attribute to the mapping element
    if mapping_element is not None:
        mapping_element.set('ref', new_value)
        tree.write(file, pretty_print=True,encoding="UTF-8")



#

In [ ]:
def has_empty_locale(obj):
    has_empty = False
    for key in obj:
        if not obj[key]:
            has_empty = True
    return has_empty

def most_frequent(List):
    return max(set(List), key = List.count)


In [ ]:
# searchStr = 'Successfully Added!' #TEST CASE
final_no_translations = []
no_matches_in_algo = []
# word_list = ['Consent Agreement']
for searchStr in word_list:
    matched_elements = []
    matched_elements_ref = []
    ref_occurence = []
    for element in resource_dictionary:
        for mapping in resource_dictionary[element]:
            if searchStr == resource_dictionary[element][mapping]['value']:
                resElement = f"{element}.{mapping}"
                element_disc = {}
                element_disc[element] = mapping
                if resource_dictionary[element][mapping]['ref']:
                    matched_elements_ref.append(element_disc)
                else:
                    matched_elements.append(element_disc)

    for element in matched_elements_ref:
        for key in element:
            ref_occurence.append(resource_dictionary[key][element[key]]['ref'])

    # print("\nMatched_Elements")
    # pprint(matched_elements)
    # print("\nMatched_Elements_Ref")
    # pprint(matched_elements_ref)
    for top_element in matched_elements:
        matched_wref = []
        no_match = []
        for top_key in top_element:
            # print("\nEK:", top_element, top_key, top_element[top_key])
            if matched_elements_ref:
                for ref_element in matched_elements_ref:
                    for ref_key in ref_element:
                        if top_element[top_key] == ref_element[ref_key]:
                            matched_wref.append(ref_element)
                        else:
                            no_match.append(ref_element)

        occurence_list = []
        # print("\nmatchedwref:")
        # pprint(matched_wref)
        # print("\nnomatch:")
        # pprint(no_match)

        if matched_wref:
            for element in matched_wref:
                for key in element:
                    occurence_list.append(
                        resource_dictionary[key][element[key]]['ref'])
            if occurence_list:
                # print("\nOccurences")
                # pprint(occurence_list)
                new_reference = most_frequent(occurence_list)
                for key in top_element:
                    for path in module_files_path_list:
                        if key.split('.')[-1] in path:
                            modify_xml(path, top_element[key], new_reference)
                            # print('\nnew reference:', new_reference,
                            #       'for in', top_element[key], 'in', path)

        else:
            try:
                if has_empty_locale(app_translation_dict[top_key][top_element[top_key]]):
                    # print("EMPTY LOCALE", top_element, top_element[top_key])
                    find = False
                    for a_element in app_translation_dict:
                        for a_key in app_translation_dict[a_element]:
                            if not find:
                                if top_element[top_key] == a_key and not has_empty_locale(app_translation_dict[a_element][a_key]):
                                    # print("FOUND MATCH WITH TRANSLATION", a_element, "=",
                                    #       app_translation_dict[a_element][a_key])
                                    find = True
                                    for key in top_element:
                                        for path in module_files_path_list:
                                            if key.split('.')[-1] in path:
                                                modify_xml(path, top_element[key], ".".join(
                                                    [a_element, a_key]))
                                                # print('modified', path)
                                        # print('FOUND',a_element,a_key)

                    if not find:
                        # print("Found No Reference Anywhere")
                        no_matches_in_algo.append(top_element)                    # print(f"No Matches for [{a_element},{a_key}]")
                    

                
                    # print("HAS TRANSLATION")
            except Exception as e:
                # print("NOT EXIST IN app_translation_dict")
                find = False
                for a_element in app_translation_dict:
                    for a_key in app_translation_dict[a_element]:
                        if not find:
                            if top_element[top_key] == a_key and not has_empty_locale(app_translation_dict[a_element][a_key]):
                                # print("FOUND MATCH WITH TRANSLATION", a_element, "=",
                                #       app_translation_dict[a_element][a_key])
                                find = True
                                for key in top_element:
                                    for path in module_files_path_list:
                                        if key.split('.')[-1] in path:
                                            modify_xml(path, top_element[key], ".".join(
                                                [a_element, a_key]))
                                            # print('modified', path)
                if not find:
                    # print("Found No Reference Anywhere")
                    no_matches_in_algo.append(top_element)


# print("\nMatched_Elements")
# pprint(matched_elements)
# print("\nMatched_Elements_Ref")
# pprint(matched_elements_ref)
# checkLocale(app_translation_dict['CMAdmin.resource.application.ApplicationFormDetailsResource']['configSuccessMsg'])
# print(app_translation_dict['CMAdmin.resource.application.ApplicationFormDetailsResource']['configSuccessMsg'])

In [ ]:
print(len(no_matches_in_algo), 'length')
json_dict = {}
for element in no_matches_in_algo:
    for key in element:
        json_dict[".".join([key,element[key]])] = resource_dictionary[key][element[key]]['value']

with open("final_output.json", 'w') as file:
    file.write(json.dumps(json_dict, indent=4))
